In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import time

In [ ]:
# !rm -r 1D-Shallow-Water-Equation
# !git clone --branch curso_unah_sesion_3 --single-branch https://github.com/erickurquilla1999/1D-Shallow-Water-Equation.git
# !mv 1D-Shallow-Water-Equation/*.py ./

import sys
from pathlib import Path

aqui_camino = Path.cwd()
cdg_fuente_camino = (aqui_camino / '..' / 'cdg_fuente').resolve()
sys.path.append(str(cdg_fuente_camino))

import bases
import galerkin_discontinuo 
import paso_de_tiempo
import graficos

In [ ]:
# Parametros de entrada

# Dominio espacial de la simulacion 
x_inicial = 0 # (m) cooordenda inicial del dominio
x_final = 10 # (m) coordenada final del dominio

# Parametros del metodo de elementos finitos
N_elementos = 6 # numero de elementos
N_nodos = 4 # numero de nodos por elemento (por simplicidad solo consideramos N_nodos >= 2)

# Dominio temporal 
n_pasos = 100/20 # numero de pasos temporales
t_total = 1/20 # (s) tiempo final

# Integracion numerica con cuadratura de Gauss
n_nodos_cuadratura_gauss = 20

In [ ]:
# Generando malla
longitud_elemento = (x_final - x_inicial) / N_elementos
malla = np.array([np.linspace(x_inicial + i * (x_final - x_inicial) / N_elementos, x_inicial + (i + 1) * (x_final - x_inicial) / N_elementos, N_nodos) for i in range(N_elementos)])      

# Condiciones iniciales
h = 1.0 + 0.1 * np.exp( - ( malla - 5.0 )**2 ) # Altura (m)
u = malla * 0.0 # Velocidad horizontal (m/s)

In [ ]:
cuadratura_de_gauss, pesos_de_gauss = np.polynomial.legendre.leggauss(n_nodos_cuadratura_gauss)
polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss = bases.generate_reference_space(malla, cuadratura_de_gauss)
matriz_de_masa_inversa = galerkin_discontinuo.calcula_inversa_matriz_de_masa(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, N_nodos)
matriz_de_rigidez = galerkin_discontinuo.calcula_matrix_de_rigidez(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss)

In [ ]:
# time step
time_step = np.array(t_total/n_pasos) 

# evolving in time the PDE
for number_of_t_step in np.arange(n_pasos):

    dh_dt, du_dt = paso_de_tiempo.compute_dhdt_du_dt(h, u, matriz_de_rigidez, matriz_de_masa_inversa)

    # inicializando los valores de h y u en el siguiente paso de tiempo
    h_nuevo = np.zeros((N_elementos, N_nodos))
    u_nuevo = np.zeros((N_elementos, N_nodos))

    # calculando los valores de h y u en el siguiente paso de tiempo con el metodo de Euler explicito
    # ... Escribe aqui h_nuevo y u_nuevo dados por el metodo de Euler explicito ...
    h_nuevo = h + dh_dt * time_step
    u_nuevo = u + du_dt * time_step
    
    # -----------------------------------------------------------------------------------------

    # graficando la simulacion
    graficos.plot_simulation(malla, h_nuevo, u_nuevo, N_elementos, time_step, number_of_t_step, display=True)

    # actualizando los valores de h y u
    h = h_nuevo
    u = u_nuevo

print(f'Done')